In [1]:
# As usual, a bit of setup
from __future__ import print_function
import time
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.fc_net import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

=========== You can safely ignore the message below if you are NOT working on ConvolutionalNetworks.ipynb ===========
	You will need to compile a Cython extension for a portion of this assignment.
	The instructions to do this will be given in a section of the notebook below.
	There will be an option for Colab users and another for Jupyter (local) users.


In [2]:
# Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in list(data.items()):
  print(('%s: ' % k, v.shape))

('X_train: ', (49000, 3, 32, 32))
('y_train: ', (49000,))
('X_val: ', (1000, 3, 32, 32))
('y_val: ', (1000,))
('X_test: ', (1000, 3, 32, 32))
('y_test: ', (1000,))


In [7]:

num_inputs = 2
input_shape = (4, 5, 6)
output_dim = 3

input_size = num_inputs * np.prod(input_shape)
weight_size = output_dim * np.prod(input_shape)

x = np.linspace(-0.1, 0.5, num=input_size).reshape(num_inputs, *input_shape)
w = np.linspace(-0.2, 0.3, num=weight_size).reshape(np.prod(input_shape), output_dim)
b = np.linspace(-0.3, 0.1, num=output_dim)
x.shape

(2, 4, 5, 6)

In [10]:
int(x.size/x.shape[0])

120

In [10]:
for i in range(2):
    print(i)

0
1


In [3]:
for i in range(6, 0, -1):
    print(i)

6
5
4
3
2
1


In [42]:
import numpy as np

In [43]:
x = np.array([[1,2,3,4,5],
              [2,3,4,5,6]])
x

array([[1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6]])

In [44]:
x[:,2]

array([3, 4])

In [45]:
np.var(x,axis=0)

array([0.25, 0.25, 0.25, 0.25, 0.25])

In [50]:
x.sum((0,1))

35

In [75]:
x = np.array([[[1,2,3,4,5],
               [2,3,4,5,6]],
              [[1,2,3,4,5],
               [2,3,4,5,6]]])
x.shape

(2, 2, 5)

In [63]:
x[1,:,2][:,None,None]

array([[[3]],

       [[4]]])

In [66]:
x[1,:,2][:,None,None] * x

array([[[ 3,  6,  9, 12, 15],
        [ 6,  9, 12, 15, 18]],

       [[ 4,  8, 12, 16, 20],
        [ 8, 12, 16, 20, 24]]])

In [73]:
x[1,:,2].reshape()

ValueError: cannot reshape array of size 2 into shape (4,1)

In [76]:
x.sum(axis=0)

array([[ 2,  4,  6,  8, 10],
       [ 4,  6,  8, 10, 12]])

In [3]:
a = np.array([[1,2,3],
              [2,3,4]])
b = np.array([[2,2,2],
              [3,3,3]])
a*b

array([[ 2,  4,  6],
       [ 6,  9, 12]])

In [14]:
t = np.array([1,2,3,4,5])
p = np.array([2,2,2])
t.shape

(5,)

In [15]:
t*p

ValueError: operands could not be broadcast together with shapes (5,) (3,) 

In [19]:
aa = t.reshape(-1,1) * p
aa

array([[ 2,  2,  2],
       [ 4,  4,  4],
       [ 6,  6,  6],
       [ 8,  8,  8],
       [10, 10, 10]])

In [33]:
np.random.rand(*aa.shape)

array([[0.92883547, 0.09686497, 0.75562604],
       [0.84286983, 0.52332112, 0.80479178],
       [0.66149298, 0.88568596, 0.86552394],
       [0.12400098, 0.53782323, 0.21235944],
       [0.54447601, 0.77776715, 0.38070256]])

In [ ]:
np.random.rand(*aa.shape) < 0.5

In [34]:
i = 1
print(i != 1)

False


In [31]:
aa * aa

array([[  4,   4,   4],
       [ 16,  16,  16],
       [ 36,  36,  36],
       [ 64,  64,  64],
       [100, 100, 100]])

In [30]:
aa - p

array([[0, 0, 0],
       [2, 2, 2],
       [4, 4, 4],
       [6, 6, 6],
       [8, 8, 8]])

In [21]:
aa * t.reshape(-1,1)

array([[ 2,  2,  2],
       [ 8,  8,  8],
       [18, 18, 18],
       [32, 32, 32],
       [50, 50, 50]])

In [28]:
aa * p

array([[ 4,  4,  4],
       [ 8,  8,  8],
       [12, 12, 12],
       [16, 16, 16],
       [20, 20, 20]])

In [22]:
import numpy as np
a = np.arange(27).reshape(3,3,3)
a

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [23]:
np.transpose(a,(1,2,0))

array([[[ 0,  9, 18],
        [ 1, 10, 19],
        [ 2, 11, 20]],

       [[ 3, 12, 21],
        [ 4, 13, 22],
        [ 5, 14, 23]],

       [[ 6, 15, 24],
        [ 7, 16, 25],
        [ 8, 17, 26]]])

In [18]:
b = np.arange(12).reshape(2,3,2)
b

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5]],

       [[ 6,  7],
        [ 8,  9],
        [10, 11]]])

In [19]:
b.reshape(-1,2)

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11]])

In [20]:
b = np.arange(12).reshape(2,2,3)
b

array([[[ 0,  1,  2],
        [ 3,  4,  5]],

       [[ 6,  7,  8],
        [ 9, 10, 11]]])

In [21]:
b.reshape(-1,2)

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11]])

In [1]:
import torch 

In [4]:
x = torch.randn(10, 5, requires_grad = True)
y = torch.randn(10, 5, requires_grad = True)
z = torch.randn(10, 5, requires_grad = True)
with torch.no_grad():
    w = x + y + z
    print(w.requires_grad)
    print(w.grad_fn)
print(w.requires_grad)
print(x.requires_grad)

False
None
False
True
